In [24]:
# simultaneous alerts problem - OPEN
# internet interruption problem - OPEN


import requests
from bs4 import BeautifulSoup as scrape
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from playsound import playsound
import time

qrtx3070 = "geforce+rtx+3070"
qrtx3060tiA = "geforce+rtx+3060+ti"
qrtx3060tiB = "geforce+rtx+3060ti"
qrtx3080 = "geforce+rtx+3080"
qrtx3090 = "geforce+rtx+3090"
qrxRadeon67 = "Radeon+RX+6700"
qrxRadeon68 = "Radeon+RX+6800"
qrxRadeon69 = "Radeon+RX+6900"

Krtx3070 = ["geforce","rtx","3070"]
Krtx3060tiA = ["geforce","rtx","3060","ti"]
Krtx3060tiB = ["geforce","rtx","3060ti"]
Krtx3080 = ["geforce","rtx","3080"]
Krtx3090 = ["geforce","rtx","3090"]
KrxRadeon67 = ["Radeon","RX","6700"]
KrxRadeon68 = ["Radeon","RX","6800"]
KrxRadeon69 = ["Radeon","RX","6900"]


pidlist =     [ 'rtx3060tiA','rtx3060tiB','rtx3070','rtx3080','rtx3090' ]                  
querylist =   [ qrtx3060tiA, qrtx3060tiB, qrtx3070, qrtx3080, qrtx3090  ] # qrxRadeon67, qrxRadeon68, qrxRadeon69 ]
keywordlist = [ Krtx3060tiA, Krtx3060tiB, Krtx3070, Krtx3080, Krtx3090  ] # KrxRadeon67, KrxRadeon68, KrxRadeon69 ]
keywordsbook = { 'rtx3060tiA': Krtx3060tiA, 'rtx3060tiB': Krtx3060tiB, 'rtx3070': Krtx3070, 'rtx3080': Krtx3080, 'rtx3090': Krtx3090 } # KrxRadeon67, KrxRadeon68, KrxRadeon69 ] 

upper_price =  {  'rtx3060tiA': 850, 'rtx3060tiB': 850, 'rtx3070': 850, 'rtx3080': 1330, 'rtx3090': 2400 }
identityPrice = { 'rtx3060tiA': 600, 'rtx3060tiB': 600 ,'rtx3070': 600, 'rtx3080':  700, 'rtx3090': 1650 } # product verification criterion

def checkNameList(name,keylist):

    target = name.upper()

    for key in keylist:
        if key.upper() not in target:
            return False

    return True

def connect_url(url):
    while True:
        try:
            r = requests.get(url, timeout=5)
            break
        except requests.exceptions.Timeout:
            print("ERROR: connection timed out, try again... ")
            
        #except requests.exceptions.ConnectionError:
            #print("ERROR: wrong url")

    if r.status_code == requests.codes.ok:
        return scrape(r.content, 'lxml')
    else: 
        return False

def initSearch_mindfactory(pid,query):
    
    #query = "rtx+3070"
    #pid = 'rtx3070'
    
    doc = connect_url("https://www.mindfactory.de/search_result.php?search_query="+query)
    blacklist = [] 

    if doc:        

        titelinfo = doc.title.text

        if titelinfo.startswith("Suche nach"):
            if ":" in titelinfo:
                itemlist = doc.find(id='bProducts').find_all(_class='p')

                for i in itemlist:
                    # check item
                    pname = i.find(class_="pname").text 
                    #                if pname not in blacklist:
                    prc = i.find(class_="pprice").text
                    prc = prc[2:-1].replace('.','').replace(',','.')
                    pprice = float(prc)

                    if checkNameList(pname,keywordsbook[pid]) and (upper_price[pid] > pprice > identityPrice[pid]):
                        alert_mindfactory( i.find('a').get("href") )

                    blacklist.append(pname)


    return blacklist
                    
def search_mindfactory(pid,query,blacklist):
    # ITERATION

    doc = connect_url("https://www.mindfactory.de/search_result.php?search_query="+query)
    blacklist_ = blacklist

    if doc:        

        titelinfo = doc.title.text

        if titelinfo.startswith("Suche nach"):
            if ":" in titelinfo:
                itemlist = doc.find(id='bProducts').find_all(_class='p')

                for i in itemlist:
                    # check item
                    pname = i.find(class_="pname").text 
                    if pname not in blacklist:
                        prc = i.find(class_="pprice").text
                        prc = prc[2:-1].replace('.','').replace(',','.')
                        pprice = float(prc)

                        if checkNameList(pname,keywordsbook[pid]) and (upper_price[pid] > pprice > identityPrice[pid]):
                            link = i.find('a').get("href")
                            doc = connect_url(link) # no check !
                            
                            # Liefercheck
                            lieferbarkeit = doc.find(id="priceCol")
                            ok = lieferbarkeit.find(id="btn-buy-productInfo")
                            if ok: 
                                alert_mindfactory(link)

                        blacklist.append(pname)


    
    return blacklist_ 


def alert_mindfactory(hyperlink):
    
    # Load chrome
    driver = webdriver.Chrome('./chromedriver/chromedriver')
    driver.get("https://www.mindfactory.de/")
    driver.find_element_by_id("checkcookie").click()
    
    driver.get(hyperlink)
    driver.find_element_by_id("btn-buy-productInfo").click()
    driver.get("https://mindfactory.de/shopping_cart.php")
    driver.get("https://www.mindfactory.de/order_login.php")
    
    mail = driver.find_element_by_id("login_email_address")
    mail.clear()
    mail.send_keys("michaelblachetta@gmail.com")
    psw = driver.find_element_by_id("login_password")
    psw.clear()
    psw.send_keys("M#yxcvbnm123")
    form = driver.find_element_by_id("login_form")
    form.submit()
    
    driver.find_element_by_name("conditions").click()
    driver.find_element_by_name("privacy").click()
    driver.find_element_by_name("disclaimer").click()
    
    driver.maximize_window()    
    playsound('alert.mp3')
    answer = input("press any key to continue... ")
    
    #if answer == 'y':
    #    bl.append(pname)
    #root = tk.Tk()
    #MsgBox = tk.messagebox.askquestion ('addBlcklist','Do you want to add the item to the blacklist ?',icon = 'warning')
    #if MsgBox == 'yes':
    #    bl.append(pname)
    
    #root.destroy()
        
    return

if __name__ == '__main__':
    
    blist = []

    for i, q in enumerate(querylist):
        blist.append( initSearch_mindfactory(pidlist[i],q) )

    while(True):
        for i, q in enumerate(querylist):
            blist[i] = search_mindfactory(pidlist[i],q,blist[i])


#blacklist.append(pname)
    


#autobuyprice = {'rtx3060ti': 750}

#else:
#    print("404 - not found")

#titelinfo
# --> 200 OK
# --> find(id="priceCol")
# --> find(id="btn-buy-productInfo") # does exist => ist lieferbar

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [25]:
doc = connect_url("https://www.mindfactory.de/product_info.php/pid/awin/24GB-INNO3D-GeForce-RTX-3090-iChill-FROSTBITE-GDDR6X-HDMI-3xDP-WK_1400592.html?utm_source=zanox&utm_medium=affiliate&utm_campaign=cpo&awc=12252_1616008083_4f0c4bd58d2d2e105295398b78c7cffb")
doc

False

In [14]:
#doc = connect_url("https://www.mindfactory.de/product_info.php/16GB-Powercolor-Radeon-RX-6800XT-Red-Devil-Edition-DDR6--Retail-_1390305.html")
doc = connect_url("https://www.mindfactory.de/product_info.php/Notebook-17-3Zoll--43-94cm--MSI-GP76-Leopard-10UG-291--144-Hz-3-ms--i7-_1397039.html")
ok = suchblock.find(id="btn-buy-productInfo")
#if doc:        
    #r = requests.get("https://www.mindfactory.de/search_result.php?search_query="+query)
    #doc = scrape(r.content, 'lxml')
    #print(doc)
    #titelinfo = doc.title.text
    #titelinfo
suchblock = doc.find(id="priceCol")

In [15]:
x = suchblock.find(id="btn-buy-productInfo")
if x:
    print('Kauf möglich')
else:
    print('nicht lieferbar')

Kauf möglich


In [16]:
identityPrice = {'rtx3070': 600, 'rtx3060ti': 600, 'rtx3080': 700, 'rtx3090': 1650} # product verification criterion

bs4.element.Tag

In [19]:
i = scrape("<div> <a href='SUPER.x' /> </div>", 'html5lib')
type(i.find('a').get("href"))

str

In [26]:

import clx.xms
client = clx.xms.Client('2f70139a045b49bda73198693be5db46', 'fa5b002cc9d344b889d9ba71b06679d0')

In [27]:
client.create_text_message(sender='123456789',recipient='491706554198', body='GeForce RTX 3060,\nPreis:658\nhttps://www.mindfactory.de/product_info.php/pid/awin/24GB-INNO3D-GeForce-RTX-3090-iChill-FROSTBITE-GDDR6X-HDMI-3xDP-WK_1400592.html?utm_source=zanox&utm_medium=affiliate&utm_campaign=cpo&awc=12252_1616008083_4f0c4bd58d2d2e105295398b78c7cffb')